# Creating DCNN training libraries from label images

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

This notebook is to demonstrate how to create libraries of categorized image tiles from ground truth (labelled) images, in order to retrain a DCNN using transfer learning

### Overview

The image is divided up into tiles of a specified size, "tilesize". 

If the proportion of pixels within the tile is greater than a specified amount, "threshold", then the tile is written to a file in a folder denoting its class. 

The tiles can then be 'thinned', by specifying what proportion of tiles (randomly selected from the entire catalogue) to keep - "prop_keep"

This generates both ground-truth label imagery (to evaluate classification performance) and sets of data suitable for training a DCNN.

### Using a local directory

The images are from the California Coastal Records Project

Copyright (C) 2002–2018 Kenneth and Gabrielle Adelman, www.Californiacoastline.org and are used with permission. 

First, let's take a look at how many test/train images we have

In [1]:
from glob import glob
files = glob('ccr_train/*.JPG')
len(files)

30

30 training images. How many test images?

In [2]:
files = glob('ccr_test/*.JPG')
len(files)

19

we have 19 training images

Ok, let's create some 224x224 pixel tiles from the training images. We'll only keep tiles with 90% or more of pixels representing the dominant class, then we'll randomly select 10% of those tiles to train with (to speed things up)

In [3]:
direc='ccr_train'
tilesize = 224
threshold = 0.9
prop_keep=0.125

Retiling takes a few minutes, so let's watch a video on one convolutional net layer

In [ ]:
from IPython.display import YouTubeVideo
## CNN example
YouTubeVideo('bXJx7y51cl0') ##11 mins

In [ ]:
## approx 6 mins

In [4]:
%run ./retile.py $direc $tilesize $threshold $prop_keep

ccr_train/201300025.JPG
ccr_train/201301034.JPG
ccr_train/201301088.JPG
ccr_train/201301127.JPG
ccr_train/201302268.JPG
ccr_train/201302501.JPG
ccr_train/201302679.JPG
ccr_train/201302765.JPG
ccr_train/201303369.JPG
ccr_train/201303464.JPG
ccr_train/201303950.JPG
ccr_train/201304247.JPG
ccr_train/201304817.JPG
ccr_train/201305224.JPG
ccr_train/201305496.JPG
ccr_train/201305596.JPG
ccr_train/201305657.JPG
ccr_train/201305721.JPG
ccr_train/201305944.JPG
ccr_train/201306308.JPG
ccr_train/201306532.JPG
ccr_train/201306933.JPG
ccr_train/201307292.JPG
ccr_train/201308459.JPG
ccr_train/201308874.JPG
ccr_train/201309043.JPG
ccr_train/201309819.JPG
ccr_train/201309922.JPG
ccr_train/201310389.JPG
ccr_train/201310440.JPG
thinning files ...
beach: 257
buildings: 322
cliff: 752
road: 1264
sky: 1487
surf: 2343
swash: 95
terrain: 568
veg: 64
water: 429


To do the same on the 'test' directory:

In [5]:
direc='ccr_test'
tilesize = 224
threshold = 0.9
prop_keep=0.125

In [6]:
## approx 4 mins

In [7]:
%run ./retile.py $direc $tilesize $threshold $prop_keep 

ccr_test/201301651.JPG
ccr_test/201302661.JPG
ccr_test/201302989.JPG
ccr_test/201303838.JPG
ccr_test/201304702.JPG
ccr_test/201305404.JPG
ccr_test/201305642.JPG
ccr_test/201305789.JPG
ccr_test/201306224.JPG
ccr_test/201306768.JPG
ccr_test/201306957.JPG
ccr_test/201308103.JPG
ccr_test/201308826.JPG
ccr_test/201308902.JPG
ccr_test/201309654.JPG
ccr_test/201309844.JPG
ccr_test/201310085.JPG
ccr_test/201310435.JPG
ccr_test/201310453.JPG
thinning files ...
beach: 218
buildings: 600
cliff: 427
road: 640
sky: 922
surf: 817
swash: 49
terrain: 543
veg: 10
water: 386


### From S3 bucket

In [11]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
S3direc='cdi-workshop/semseg_data/ontario/train'
print(S3direc)

cdi-workshop/semseg_data/ontario/train


How many files?

In [12]:
files = [f for f in fs.ls(S3direc) if f.endswith('.JPG')]
len(files)

24

In [13]:
tilesize = 224
threshold = 0.9
prop_keep=0.25

The s3 bucket is a read-only filesystem, which means we have to write out the tiles here to your local directory

In [ ]:
from IPython.display import YouTubeVideo
## Why convolutions?
YouTubeVideo('ay3zYUeuyhU') ##9 mins

In [14]:
%run ./retile_fromS3.py $S3direc $tilesize $threshold $prop_keep

cdi-workshop/semseg_data/ontario/train
0
cdi-workshop/semseg_data/ontario/train/A2013170_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013473_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013540_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013594_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013649_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013657_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013671_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013697_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013709_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013733_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013743_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013773_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013900_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2013960_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2014128_geotag.JPG
cdi-workshop/semseg_data/ontario/train/A2014153_geotag.JPG
cdi-workshop/se

Now we are ready to retrain the DCNN

## DL-tools

The equivalent function in DL-tools is called using:

```python create_library\retile.py -t 96 -a 0.9 -b 0.5```

where ```t``` is the tile size, ```a``` is the threshold proportion of pixels in a tile uniformly classified, and ```b``` is the proportion of the tiles to randomly select and keep

You will be prompted to select a set of .mat files. The tiles will be created in folders called "test" and "train"